In [6]:
using NBInclude
using FFTW
@nbinclude("Eulergrid.ipynb")

##########Different types of operators that are to be used in IBM###########

abstract type Abstractoperators end

struct differentialoperator <:Abstractoperators
    grid::PeriodicEulergrid                #####Contains our Euler grid and frequency multplies i.e 2.pi.i.freq.
    Frequencymultiples::Matrix{Complex}
end

struct Gradientoperator <:Abstractoperators
    grid::PeriodicEulergrid
    UxFrequencymultiples::Matrix{Complex}
    VyFrequencymultiples::Matrix{Complex}
end
struct Divergenceoperator <:Abstractoperators
    grid::PeriodicEulergrid
    UxFrequencymultiples::Matrix{Complex}
    VyFrequencymultiples::Matrix{Complex}
end
####Frequencies for different operators that are to be used in FFT calculation.
function Laplacianoperator(grid::PeriodicEulergrid)
    ωx=FFTW.fftfreq(grid.Nx,grid.Nx/grid.L)
    ωy=FFTW.fftfreq(grid.Ny,grid.Ny/grid.B)
    complexfreqx=im*2*pi*ωx
    complexfreqy=im*2*pi*ωy
    ΩX = [complexfreqx[i] for i=1:grid.Nx,j=1:grid.Ny]; 
    ΩY = [complexfreqy[j] for i=1:grid.Nx,j=1:grid.Ny];
    LaplaceFFT=(ΩX.^2 + ΩY.^2)
    Laplaciandata=differentialoperator(grid, LaplaceFFT)
    return Laplaciandata
end

function Divergenceoperator(grid::PeriodicEulergrid)
    ωx=FFTW.fftfreq(grid.Nx,grid.Nx/grid.L)
    ωy=FFTW.fftfreq(grid.Ny,grid.Ny/grid.B)
    complexfreqx=im*2*pi*ωx
    complexfreqy=im*2*pi*ωy
    ΩX = [complexfreqx[i] for i=1:grid.Nx,j=1:grid.Ny]; 
    ΩY = [complexfreqy[j] for i=1:grid.Nx,j=1:grid.Ny];
    Divergencedata= Divergenceoperator(grid,ΩX,ΩY)
    return Divergencedata
end 
function Gradientoperator(grid::PeriodicEulergrid)
     ωx=FFTW.fftfreq(grid.Nx,grid.Nx/grid.L)
    ωy=FFTW.fftfreq(grid.Ny,grid.Ny/grid.B)
    complexfreqx=im*2*pi*ωx
    complexfreqy=im*2*pi*ωy
    ΩX = [complexfreqx[i] for i=1:grid.Nx,j=1:grid.Ny]; 
    ΩY = [complexfreqy[j] for i=1:grid.Nx,j=1:grid.Ny];
    Gradientdata= Gradientoperator(grid,ΩX,ΩY)
    return Gradientdata
end
##Now applying these operators to get the DFT and IDFT of data
##We have scalar data then we apply and invert DFT.
function Applydifferentialoperator(data::coordinatedata,operator::differentialoperator)
	if ~(data.grid === operator.grid)
		println("The size is not same for two matrices.")
	end
	oldhat = FFTW.fft(data.U);
	newhat = operator.Frequencymultiples.*oldhat;
	new = real(FFTW.ifft(newhat));
	result = coordinatedata(new,myoperator.grid);
	return result
end
function Applydifferentialoperator(data::Matrix,operator::differentialoperator)
	if ~(size(data)=== operator.grid)
		println("The size is not same.")
	end
	oldhat = FFTW.fft(data);
	newhat = operator.Frequencymultiples.*oldhat;
	new = real(FFTW.ifft(newhat));
	
	return new
end
## Now we will apply these operators to vector data
function Applydifferentialoperator(data::Vectordata,operator::differentialoperator)
	if ~(data.grid === operator.grid)
		println("The size is not same.")
	end
	oldUhat = FFTW.fft(data.U);
	newUhat = operator.Frequencymultiples.*oldUhat;
	newU = real(FFTW.ifft(newUhat));
    oldVhat = FFTW.fft(data.V);
	newVhat = operator.Frequencymultiples.*oldVhat;
	newV = real(FFTW.ifft(newVhat));
	result = Vectordata(newU,newV,operator.grid);
	return result
end

function Applydifferentialoperator(dataU::Matrix,dataV::Matrix,operator::differentialoperator)
	if ~(size(dataU)=== (operator.grid.Nx,operator.grid.Ny))
		println("The size is not same for two matrices.")
	end
    if ~(size(dataV)=== (operator.grid.Nx,operator.grid.Ny))
		println("The size is not same for two matrices.")
	end
	oldUhat = FFTW.fft(dataU);
	newUhat = operator.Frequencymultiples.*oldUhat;
	newU = real(FFTW.ifft(newUhat));
    oldVhat = FFTW.fft(dataV);
	newVhat = operator.Frequencymultiples.*oldVhat;
	newV = real(FFTW.ifft(newVhat));
	result = Vectordata(newU,newV,operator.grid);
	return newU,newV
end
#####Applying divergence operator.
function ApplyDivergenceoperator(data::Vectordata,operator::Divergenceoperator)
	if ~(data.grid === operator.grid)
		println("The size is not same.")
	end
	oldUhat = FFTW.fft(data.U);
	oldVhat = FFTW.fft(data.V);
	newUhat = operator.UxFrequencymultiples.*oldUhat;
	newVhat = operator.VyFrequencymultiples.*oldVhat;
	new = real(FFTW.ifft(newUhat .+ newVhat));
	result = coordinatedata(new,operator.grid);
	return result
end
#Now apply to vector data stored as two arrays
function ApplyDivergenceoperator(dataU::Matrix{Float64},dataV::Matrix{Float64},operator::Divergenceoperator)
	if ~(size(dataU) == (operator.grid.Nx,operator.grid.Ny))
		println("The size is not same.")
	elseif ~(size(dataV) == (operator.grid.Nx,operator.grid.Ny))
		println("The size is not same.")
	end
	oldUhat = FFTW.fft(dataU);
	oldVhat = FFTW.fft(dataV);
	newUhat = operator.UxFrequencymultiples.*oldUhat;
	newVhat = operator.VyFrequencymultiples.*oldVhat;
	new = real(FFTW.ifft(newUhat .+ newVhat));
	return new
end
#####Finally, the gradient-like opeators#####
#####First we do scalar structs (return vector struct)#####
function ApplyGradientoperator(data::coordinatedata,operator::Gradientoperator)
	if ~(data.grid === operator.grid)
		println("The size is not same.")
	end
	oldUhat = FFTW.fft(data.U);
	newUhat = operator.UxFrequencymultiples.*oldUhat;
	newVhat = operator.VyFrequencymultiples.*oldUhat;
	newU = real(FFTW.ifft(newUhat));
	newV = real(FFTW.ifft(newVhat));
	result = Vectordata(newU,newV,operator.grid);
	return result
end

#Now apply to vector data stored as two arrays
function ApplyGradientoperator(dataU::Matrix{Float64},operator::Gradientoperator)
	if ~(size(dataU) === operator.grid.Nx,operator.grid.Ny)
		println("The size is not same.")
	end
	oldUhat = FFTW.fft(dataU);
	newUhat = operator.UxFrequencymultiples.*oldUhat;
	newVhat = operator.VyFrequencymultiples.*oldUhat;
	newU = real(FFTW.ifft(newUhat));
	newV = real(FFTW.ifft(newVhat));
	
	return newU, newV
end
function Invertlaplacianoperator(dataU::Matrix{ComplexF64},grid::PeriodicEulergrid)
    if ~(size(dataU) === (grid.Nx,grid.Ny))
		println("The size is not same for two matrices.")
	end
    Eigenvalues=Laplacianoperator(grid).Frequencymultiples
    output_trans=zeros(ComplexF64,grid.Nx,grid.Ny)
    i=1;
    j=1;
    k=1;
     output_trans = dataU./Eigenvalues;
     output_trans[1,1] = 0;
  # for i in 1:grid.Nx
   #         for j in 2:grid.Ny
    #            output_trans[i,j]=dataU[i,j]/Eigenvalues[i,j]
     #   end
    #end
    #for k in 2:grid.Nx
     #   output_trans[k,1]=dataU[k,1]/Eigenvalues[k,1]
    #end
    output=real(FFTW.ifft(output_trans))
    return output
end

Invertlaplacianoperator (generic function with 1 method)